In [1]:
!pip install -q --upgrade llama-index transformers accelerate bitsandbytes
!pip install -q llama-index-llms-huggingface llama-index-embeddings-huggingface
!pip install -q chromadb llama-index-vector-stores-chroma
!pip install -q sentence-transformers



!pip install-q  pypdf
!pip install -q tqdm
!pip install -q --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 27 08:26:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   32C    P0             55W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install pypdf
!pip install -q tqdm
!pip install --upgrade openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Y
Token is valid (permission: fineGrained).
The token `llms` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [ ]:
DOCX_PATH = "/content/drive/MyDrive/DTCNTT/DMS-5.pdf"
CACHE_FILE = "/content/drive/MyDrive/DTCNTT/data/cache/pipeline_cache.json" # Cache mới
INDEX_STORAGE = "/content/drive/MyDrive/DTCNTT/data/index_store_V6_maximalist" # Thư mục Index mới
COLLECTION_NAME = "dsm5_V6_store" # Collection mới

In [ ]:

import os
import openai

client = openai.OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [2]:
import os
import torch
import chromadb

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
)

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import (
    TitleExtractor,
    KeywordExtractor,
    SummaryExtractor,
)

from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.vector_stores.chroma import ChromaVectorStore


In [ ]:
def setup_index_models():
    global embed_model, llm_gptmini

    # =====================================================
    # 1) Load Embedding (CẦN CHO VECTOR)
    # =====================================================
    print("⏳ Đang tải Embedding model...")
    EMBEDDING_MODEL = "AITeamVN/Vietnamese_Embedding"
    embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
    Settings.embed_model = embed_model
    print("✅ Embedding đã sẵn sàng!\n")

    # =====================================================
    # 2) Load GPT-4o-mini (TRANSFORMATION)
    # =====================================================
    print("⏳ Đang tải GPT-4o-mini (cho Title/Keyword/Summary)…")
    llm_gptmini = OpenAI(
        model="gpt-4o-mini",
        temperature=0.3,
        max_tokens=512
    )
    print("✅ GPT-4o-mini đã sẵn sàng!\n")

    print("🎉 MÔI TRƯỜNG GIAI ĐOẠN 1 SẴN SÀNG: Embedding + GPT-4o-mini")

    return embed_model, llm_gptmini


In [ ]:
embed_model, llm_gptmini = setup_index_models()
Settings.llm = llm_gptmini           # chỉ dùng cho transformation
Settings.embed_model = embed_model


⏳ Đang tải Embedding model...
✅ Embedding đã sẵn sàng!

⏳ Đang tải GPT-4o-mini (cho Title/Keyword/Summary)…
✅ GPT-4o-mini đã sẵn sàng!

🎉 MÔI TRƯỜNG GIAI ĐOẠN 1 SẴN SÀNG: Embedding + GPT-4o-mini


In [ ]:
# ================================================================
# 2. INGEST TÀI LIỆU + METADATA (Title/Keyword/Summary) + AUGMENT
# ================================================================
def ingest_documents():

    print("📂 (Giai đoạn 1) Đọc tài liệu ...")
    if not os.path.exists(DOCX_PATH):
        print(f"❌ LỖI: File không tồn tại: {DOCX_PATH}")
        return []

    documents = SimpleDirectoryReader(
        input_files=[DOCX_PATH],
        filename_as_id=True,
    ).load_data()
    print(f"✅ Đã tải {len(documents)} tài liệu.\n")
    if not documents:
        return []

    # Cache ingest
    try:
        cached_hashes = IngestionCache.from_persist_path(CACHE_FILE)
        print("🧠 (Giai đoạn 1) Dùng lại cache.\n")
    except Exception:
        cached_hashes = None
        print("ℹ️ (Giai đoạn 1) Không có cache, chạy mới.\n")

    # Transformation LLM = GPT-4o-mini
    llm_transform = llm_gptmini
    print("🤖 Transformation LLM = GPT-4o-mini\n")

    # Sentence splitter
    splitter = SentenceSplitter(
        chunk_size=512,
        chunk_overlap=20,
    )

    transformations = [
        splitter,
        TitleExtractor(nodes=5, llm=llm_transform),
        KeywordExtractor(keywords=10, llm=llm_transform),
        SummaryExtractor(summaries=["prev", "self"], llm=llm_transform),
    ]

    print("🚀 (Giai đoạn 1) Đang chạy pipeline + embedding...\n")

    pipeline = IngestionPipeline(
        transformations=[
            *transformations,
            Settings.embed_model,  # cuối cùng là embedding
        ],
        cache=cached_hashes,
    )

    nodes = pipeline.run(documents=documents)
    print(f"✅ (Giai đoạn 1) Đã tạo {len(nodes)} nodes.\n")

    # 🔥 AUGMENT METADATA CHO HYBRID (BM25 + SEMANTIC)
    print("🧩 (Giai đoạn 1) Đang thêm metadata bm25_text + semantic_text...")

    for node in nodes:
        title = node.metadata.get("title", "")
        keywords = node.metadata.get("keywords", [])
        summary = node.metadata.get("summary", "")

        if isinstance(keywords, list):
            keyword_str = ", ".join(keywords)
        else:
            keyword_str = str(keywords)

        # Text cho BM25 (lexical)
        bm25_text = (
            node.text
            + f"\n\nTITLE: {title}"
            + f"\nKEYWORDS: {keyword_str}"
        )

        # Text cho semantic embedding
        semantic_text = (
            node.text
            + f"\n\nSUMMARY: {summary}"
        )

        node.metadata["bm25_text"] = bm25_text
        node.metadata["semantic_text"] = semantic_text

    print("✅ (Giai đoạn 1) Metadata augmentation hoàn tất.\n")

    pipeline.cache.persist(CACHE_FILE)
    print(f"💾 (Giai đoạn 1) Đã lưu cache vào {CACHE_FILE}")

    return nodes


In [ ]:
def build_index_with_chroma(nodes):
    """Xây Index và lưu vào ChromaDB"""
    print("⏳ (Giai đoạn 1) Đang tạo Index với ChromaDB...")

    if not nodes:
        raise ValueError("Không có nodes để tạo Index.")

    os.makedirs(INDEX_STORAGE, exist_ok=True)
    db = chromadb.PersistentClient(path=INDEX_STORAGE)

    chroma_collection = db.get_or_create_collection(COLLECTION_NAME)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

    print("✅ (Giai đoạn 1) Index (V6 - Siêu Metadata) đã tạo và lưu vào ChromaDB.")
    return vector_index

In [ ]:
# GIAI ĐOẠN 1: Setup + Ingest + Build Index
nodes = ingest_documents()
vector_index = build_index_with_chroma(nodes)


📂 (Giai đoạn 1) Đọc tài liệu ...
✅ Đã tải 106 tài liệu.

ℹ️ (Giai đoạn 1) Không có cache, chạy mới.

🤖 Transformation LLM = GPT-4o-mini

🚀 (Giai đoạn 1) Đang chạy pipeline + embedding...



100%|██████████| 325/325 [06:44<00:00,  1.24s/it]


✅ (Giai đoạn 1) Đã tạo 325 nodes.

🧩 (Giai đoạn 1) Đang thêm metadata bm25_text + semantic_text...
✅ (Giai đoạn 1) Metadata augmentation hoàn tất.

💾 (Giai đoạn 1) Đã lưu cache vào /content/drive/MyDrive/DTCNTT/data/cache/pipeline_cache.json
⏳ (Giai đoạn 1) Đang tạo Index với ChromaDB...
✅ (Giai đoạn 1) Index (V6 - Siêu Metadata) đã tạo và lưu vào ChromaDB.


In [ ]:
db = chromadb.PersistentClient(path=INDEX_STORAGE)
col = db.get_collection(COLLECTION_NAME)
print("📦 Số lượng vectors trong Chroma:", col.count())


📦 Số lượng vectors trong Chroma: 325


In [ ]:
def print_page(page_number):
    print(f"\n==================== PAGE {page_number} ====================\n")

    db = chromadb.PersistentClient(path=INDEX_STORAGE)
    col = db.get_collection(COLLECTION_NAME)

    results = col.get(
        where={"page_label": str(page_number)},
        include=["documents", "metadatas"]
    )

    if not results["documents"]:
        print("⚠️ Không tìm thấy trang này trong Chroma.")
        return

    text = results["documents"][0]
    meta = results["metadatas"][0]

    # In text
    print("📄 **NỘI DUNG TRANG:**")
    print("-" * 60)
    print(text[:1500])   # tránh in quá dài
    print("-" * 60)

    # In metadata
    print("\n🔖 **METADATA:**")
    for key, val in meta.items():
        print(f"  • {key}: {val}")

    print("\n============================================================\n")


In [ ]:
print_page(50)




==================== PAGE 50 ====================

📄 **NỘI DUNG TRANG:**
------------------------------------------------------------
C. Các triệu chứng ám ảnh cưỡng bức là không phải do tác dụng sinh lý của một
chất (ví dụ, ma túy, thuốc) hoặc một bệnh khác.
D. Rối loạn này không phải là các triệu chứng của rối loạn tâm thần khác (ví dụ,
lo lắng quá mức, như trong rối loạn lo âu; mối bận tâm với hình thể , như trong
cơ thể rối loạn sợ dị hình, hành vi ăn nghi thức, như trong các rối l oạn ăn, cờ
bạc, như trong các rối loạn liên quan đến chất gây nghiện; phổ tâm thần  phân
liệt và rối loạn tâm thần khác, hoặc của hành vi định hình, như trong  rối loạn tự
kỷ).
Chẩn đoán phân biệt:
* Rối loạn lo âu.
 Suy nghĩ tái diễn, hành vi né tránh, lặp đi lặp lại cho yên tâm cũng có thể xảy ra
trong các rối loạn lo âu.
------------------------------------------------------------

🔖 **METADATA:**
  • ref_doc_id: /content/drive/MyDrive/DTCNTT/DMS-5.pdf_part_49
  • file_size: 1303004
  • creation_date